In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# get Max's words
terms_file = '/home/jvdzwaan/data/adh/word-lists/farada.txt'
#terms_file = '/home/jvdzwaan/data/adh/word-lists/wajaba.txt'
terms = pd.read_csv(terms_file, encoding='utf-8', index_col=None, header=None)
terms

In [ ]:
t = terms[0].tolist()
print('total number of terms:', len(t))
terms = set(t)
print('number of unique terms:', len(terms))

In [ ]:
# root farada
root = 'فرض'

In [ ]:
# root wajaba
root = ''

In [ ]:
from lxml import etree
from tqdm import tqdm

def stemmer_xml2df2(fname):
    result = []
    
    # Extract the words
    context = etree.iterparse(fname, events=('end', ), tag=('word'))
    for event, elem in context:
        stem = None
        for a in elem.getchildren():
            if a.tag == 'analysis':
                stem = a.attrib['stem']
        result.append({'word': elem.attrib['value'], 'proposed_root': stem})
        
        # make iteration over context fast and consume less memory
        #https://www.ibm.com/developerworks/xml/library/x-hiperfparse
        elem.clear()
        while elem.getprevious() is not None:
            del elem.getparent()[0]
    
    return pd.DataFrame(result)

def analyzer_xml2df2(fname):
    result = []
    
    # Extract the words
    context = etree.iterparse(fname, events=('end', ), tag=('word'))
    for event, elem in tqdm(context):
        word = elem.attrib['value']
        #print(repr(word))
        if word != '':
            roots = []
            for a in elem.getchildren():
                if a.tag == 'analysis':
                    try:
                        roots.append(a.attrib['root'])
                    except:
                        pass
            roots = list(set(roots))
            if len(roots) == 0:
                roots.append('NOANALYSIS')
            result.append({'word': elem.attrib['value'], 'proposed_root': '\\'.join(roots)})
        
        # make iteration over context fast and consume less memory
        #https://www.ibm.com/developerworks/xml/library/x-hiperfparse
        elem.clear()
        while elem.getprevious() is not None:
            del elem.getparent()[0]
    
    return pd.DataFrame(result)

In [ ]:
import os

def root_correct(row, root):
    proposed_roots = set(row['proposed_root'].split('\\'))
    return root in proposed_roots

def get_terms(txt_file):
    # get the terms list
    terms = pd.read_csv(terms_file, encoding='utf-8', index_col=None, header=None)
    t = terms[0].tolist()
    print('total number of terms:', len(t))
    terms = set(t)
    print('number of unique terms:', len(terms))
    return terms

def match_roots(data, terms, root, regex=False):
    print('number of tokens: ', data.shape[0])
    
    # TODO: differentiate between token matches and regex search
    # Now it is token matching
    data['root'] = data.apply(lambda row: row['word'] in terms, axis=1)
    data['proposed_root_correct'] = data.apply(lambda row: root_correct(row, root), axis=1)
    data['overlap'] = (data['root'] == True) & (data['proposed_root_correct'] == True)
    result = {
        'root': root,
        '# tokens': data.shape[0],
        'semi-automatic root matches': data['root'].sum(),
        'automatically extracted root matches': data['proposed_root_correct'].sum(),
        'overlap': data['overlap'].sum(),
        'document': os.path.basename(xml_file)
    }
    
    metadata = pd.DataFrame.from_records([result], index='document')
    matches = data[(data['proposed_root_correct'] == True) | (data['root'] == True)]
    
    return metadata, matches, data

## Khoja data

In [ ]:
%%time
xml_file = '/home/jvdzwaan/data/tmp/adh/stemmer/0483IbnAhmadSarakhsi.Mabsut.xml'
terms_file = '/home/jvdzwaan/data/adh/word-lists/farada.txt'

terms = get_terms(terms_file)
khoja_data = stemmer_xml2df2(xml_file)

In [ ]:
khoja_data.shape

In [ ]:
%%time
khoja_metadata, khoja_matches, khoja_data = match_roots(khoja_data, terms, root)

In [ ]:
khoja_metadata

In [ ]:
khoja_matches.to_csv('0483IbnAhmadSarakhsi.Mabsut-khoja-farada.csv', index=None, encoding='utf-8')

## AlKhalil data

In [ ]:
%%time
xml_file = '/home/jvdzwaan/data/tmp/adh/big-xml/0483IbnAhmadSarakhsi.Mabsut.xml'
terms_file = '/home/jvdzwaan/data/adh/word-lists/farada.txt'

terms = get_terms(terms_file)
alk_data = analyzer_xml2df2(xml_file)

In [ ]:
%%time
alk_metadata, alk_matches, alk_data = match_roots(alk_data, terms, root)

In [ ]:
alk_metadata

In [ ]:
alk_metadata.to_csv('0483IbnAhmadSarakhsi.Mabsut-alkhalil-farada.csv', index=None, encoding='utf-8')